In [23]:
endpoint = 'http://50f50db3-ce0c-4d72-bc04-653e319f8723.southeastasia.azurecontainer.io/score' #Replace with your endpoint
key = 'LvalDhJvRqMg26KsehDKwYiawGCMLCDi' #Replace with your key

import json
import requests
import pandas as pd
import numpy as np

test = pd.read_csv('test.csv')
relevant_cols = ['avg_bearing', 'day of week',
       'is_Weekday', 'hour', 'time_group', 
       'origin_avg_daily_rainfall', 'dest_avg_daily_rainfall', 'origin_lat',
       'origin_lng', 'dest_lat', 'dest_lng', 'origin_region',
       'origin_subregion', 'dest_region', 'dest_subregion', 'euclid_dist']

In [20]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5600 entries, 0 to 5599
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 5600 non-null   int64  
 1   trj_id                     5600 non-null   int64  
 2   day                        5600 non-null   int64  
 3   month                      5600 non-null   int64  
 4   osname                     5600 non-null   object 
 5   avg_speed                  5600 non-null   float64
 6   avg_bearing                5600 non-null   float64
 7   hour                       5600 non-null   int64  
 8   day of week                5600 non-null   int64  
 9   is_Weekday                 5600 non-null   int64  
 10  time_group                 5600 non-null   object 
 11  origin_lat                 5600 non-null   float64
 12  origin_lng                 5600 non-null   float64
 13  dest_lat                   5600 non-null   float

In [24]:
X = test[relevant_cols]
y = test.iloc[:,-1]
ids = test['trj_id'].tolist()
y_true = y.tolist()

In [25]:
X_input = X.values.tolist()

#Convert the array to JSON format
input_json = json.dumps({"data": X_input})

#Set the content type and authentication for the request
headers = {"Content-Type":"application/json",
        "Authorization":"Bearer " + key}

#Send the request
response = requests.post(endpoint, input_json, headers=headers)

In [26]:
y_pred = []
#If we got a valid response, display the predictions
if response.status_code == 200:
    output = json.loads(response.json())
    print("Predictions:")
    for i in range(len(X_input)):
        #print (" trj_id: {}. Predicted duration: {}".format(ids[i], y["result"][i]))
        y_pred.append(output["result"][i])
else:
    print(response)

<Response [502]>


In [84]:
df = pd.DataFrame(list(zip(ids, y_pred, y_true)), columns=["trj_id","predicted_duration", "actual_duration"])

,trj_id,predicted_duration,actual_duration
0,68460,1123.966951,846
1,72213,1121.833091,1089
2,68991,1049.080585,779
3,75975,1102.002960,1002
4,64210,1316.383183,1296


In [86]:
predictions = df['predicted_duration'].values
actual = df['actual_duration'].values

array([1123.96695061, 1121.83309079, 1049.08058493, ..., 1058.14430586,
       1455.96094058, 1118.96482122])

In [88]:
from sklearn.metrics import mean_squared_error as mse

In [89]:
np.sqrt(mse(actual, predictions))

243.58166074886265